## O Algorítmo

Há duas formas de alcançar o resultado.

- Através de autovalores
- Através de uma iteração de dotproducts

O primeiro é muito custoso computacionalmente, e o segundo é uma aproximação do primeiro. Embora não encontrem valores idênticos, as primeiras posições tendem a manter a mesma ordem.